In [2]:
!pip3 install beautifulsoup4

You should consider upgrading via the 'c:\users\j_mar\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [8]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    #https://www.amazon.com.br/gp/bestsellers/books?ref_=nav_em__books_best_0_2_22_4
    r = requests.get('https://www.amazon.com.br/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [12]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor','Avaliacao','Customers_Rated', 'Preco'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [13]:
df = pd.read_csv("amazon_products.csv")
df.shape

(100, 5)

In [14]:
df.head(61)

,Titulo,Autor,Avaliacao,Customers_Rated,Preco
0,Perdida (Vol. 1): Um amor que ultrapassa as ba...,Carina Rissi,"4,7 de 5 estrelas",2.397,"R$ 14,90"
1,Vivendo Como Um Guerreiro,Whindersson Nunes,-1,0.000,"R$ 34,90"
2,Champion: Do caos e da lenda surgirá um campeão,Marie Lu,"4,7 de 5 estrelas",404.000,"R$ 20,56"
3,Prodigy: Os opostos perto do caos,Marie Lu,"4,7 de 5 estrelas",471.000,"R$ 34,12"
4,Achados e perdidos,Stephen King,"4,8 de 5 estrelas",595.000,"R$ 19,90"
...,...,...,...,...,...
56,Duna: livro 1,Frank Herbert,"4,8 de 5 estrelas",7.343,"R$ 59,90"
57,Diário de um Banana 16: Bola Fora,Jeff Kinney,"4,8 de 5 estrelas",1.540,"R$ 33,90"
58,O livro das virtudes para crianças,William Bennett,"4,8 de 5 estrelas",1.048,"R$ 34,90"
59,A morte é um dia que vale a pena viver: E um e...,Ana Claudia Quintana Arantes,"4,8 de 5 estrelas",6.047,"R$ 26,90"
